https://github.com/nn116003/torchtext-tutorial/blob/master/%E8%87%AA%E5%88%86%E3%81%AE%E3%83%87%E3%83%BC%E3%82%BF%E3%81%A7%E3%82%B5%E3%82%AF%E3%83%83%E3%81%A8attention%E3%81%A4%E3%81%8D%E6%96%87%E6%9B%B8%E5%88%86%E9%A1%9E%E3%83%A2%E3%83%87%E3%83%AB%E3%82%92%E3%81%9F%E3%82%81%E3%81%99.ipynb

In [1]:
import janome
from janome.tokenizer import Tokenizer

import torchtext
from torchtext import data
from torchtext import datasets
from torchtext.vocab import FastText

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch
from torch.autograd import Variable

torch.manual_seed(0)
torch.cuda.manual_seed(0)

In [2]:
j_t = Tokenizer()
def tokenizer(text): 
    return [tok for tok in j_t.tokenize(text, wakati=True)]

TEXT = data.Field(sequential=True, tokenize=tokenizer, lower=True, include_lengths=True, batch_first=True)
LABEL = data.Field(sequential=False, use_vocab=False)

In [67]:
# train, val, test = data.TabularDataset.splits(
#         path='./Data/', train='train.tsv',
#         validation='valid.tsv', test='test.tsv', format='tsv',
#         fields=[('Text', TEXT), ('Label', LABEL)])
train, val, test = data.TabularDataset.splits(
        path='./Data/', train='train_small.tsv',
        validation='valid_small.tsv', test='test_small.tsv', format='tsv',
        fields = [('Text', TEXT), ('Label', LABEL)])

In [51]:
vars(train[0])

{'Label': 'Label', 'Text': ['text']}

In [33]:
print('len(train)', len(train))
print('vars(train[0])', vars(train[0]))

print('len(val)', len(val))
print('vars(val[0])', vars(val[0]))

print('len(test)', len(test))
print('vars(test[0])', vars(test[0]))

len(train) 1045
vars(train[0]) {'Text': ['text'], 'Label': 'Label'}
len(val) 350
vars(val[0]) {'Text': ['text'], 'Label': 'Label'}
len(test) 350
vars(test[0]) {'Text': ['text'], 'Label': 'Label'}


In [36]:
TEXT.build_vocab(train, min_freq=10)

In [37]:
TEXT.vocab.freqs

Counter({'text': 1,
         '嫉妬': 44,
         'で': 14709,
         '彼': 805,
         'を': 21522,
         '束縛': 10,
         'し': 10957,
         'て': 19573,
         'い': 3218,
         'ませ': 620,
         'ん': 2213,
         'か': 3400,
         '？': 2070,
         'の': 35768,
         '行動': 103,
         'が': 21321,
         '気': 701,
         'に': 24249,
         'なる': 1849,
         '。': 24740,
         '携帯': 200,
         '繋がら': 2,
         'なけれ': 157,
         'ば': 1668,
         'なおのこと': 2,
         '心配': 73,
         '不安': 134,
         'は': 20809,
         '、': 34560,
         '愛し': 53,
         'いる': 5656,
         'ゆえに': 10,
         '妄想': 99,
         '働かせる': 1,
         'から': 3879,
         'だ': 5441,
         '「': 10451,
         'と': 13763,
         '連絡': 115,
         'とれ': 14,
         'ない': 5997,
         'とき': 748,
         '朝': 67,
         'まで': 1145,
         '眠れ': 15,
         'です': 3458,
         '」': 10458,
         'という': 2777,
         '萌': 6,
         'さん

In [38]:
TEXT.vocab.stoi

defaultdict(<function torchtext.vocab._default_unk_index>,
            {'<unk>': 0,
             '<pad>': 1,
             'の': 2,
             '、': 3,
             '。': 4,
             'に': 5,
             'を': 6,
             'が': 7,
             'は': 8,
             'て': 9,
             'で': 10,
             'と': 11,
             'た': 12,
             'し': 13,
             '」': 14,
             '「': 15,
             ' ': 16,
             'も': 17,
             'な': 18,
             'ない': 19,
             'いる': 20,
             'だ': 21,
             'する': 22,
             'こと': 23,
             '）': 24,
             '（': 25,
             '・': 26,
             'から': 27,
             '！': 28,
             'です': 29,
             'か': 30,
             'さ': 31,
             'い': 32,
             'や': 33,
             '人': 34,
             'ある': 35,
             'さん': 36,
             'という': 37,
             'れ': 38,
             'ます': 39,
             'う': 40,
             'よう': 41,
       

In [39]:
TEXT.vocab.itos

['<unk>',
 '<pad>',
 'の',
 '、',
 '。',
 'に',
 'を',
 'が',
 'は',
 'て',
 'で',
 'と',
 'た',
 'し',
 '」',
 '「',
 ' ',
 'も',
 'な',
 'ない',
 'いる',
 'だ',
 'する',
 'こと',
 '）',
 '（',
 '・',
 'から',
 '！',
 'です',
 'か',
 'さ',
 'い',
 'や',
 '人',
 'ある',
 'さん',
 'という',
 'れ',
 'ます',
 'う',
 'よう',
 'ん',
 '■',
 '？',
 'なっ',
 'なる',
 'できる',
 'など',
 'ば',
 '-',
 '歳',
 '『',
 '』',
 '.',
 '：',
 '2',
 '女性',
 'この',
 'たい',
 '女',
 '自分',
 'ので',
 '1',
 '”',
 '“',
 '…',
 'だろ',
 '日',
 '年',
 '3',
 'れる',
 'ね',
 '的',
 'まし',
 'その',
 'よ',
 'でも',
 '結婚',
 'だけ',
 'まで',
 'そう',
 'なく',
 'お',
 'いい',
 '月',
 'たり',
 '私',
 'もの',
 '中',
 'たら',
 'なら',
 'として',
 'より',
 'ため',
 '男性',
 '4',
 '(',
 'あり',
 'でき',
 '】',
 '5',
 '【',
 '彼',
 'って',
 'たち',
 '多い',
 'それ',
 'とき',
 'だっ',
 '今',
 'これ',
 '機能',
 'iphone',
 '気',
 '何',
 '写真',
 '/',
 'けど',
 '時',
 '独',
 'いう',
 'かも',
 '仕事',
 '見',
 'き',
 '一',
 '製品',
 'pc',
 'み',
 '方',
 'なり',
 'へ',
 '円',
 'ませ',
 '見る',
 'アプリ',
 '〜',
 'ず',
 '場合',
 'そんな',
 '思っ',
 'サービス',
 '通',
 '男',
 '好き',
 '対応',
 '思う',
 '者',
 'しまう',
 '相手',
 'また

FastTextの学習済みデータを利用する場合

In [52]:
TEXT.build_vocab(train, vectors = FastText(language="ja"), min_freq = 10)

.vector_cache/wiki.ja.vec: 1.37GB [04:57, 4.62MB/s]                               
100%|█████████▉| 579626/580000 [01:10<00:00, 11184.93it/s]

In [53]:
train_iter, val_iter, test_iter = data.Iterator.splits(
        (train, val, test), batch_sizes=(32, 256, 256), device=-1, repeat=False,sort=False)

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [54]:
batch = next(iter(train_iter))
print(batch.Text)
print(batch.Label)

(tensor([[2986,   16, 2131,  ...,  129,   12,    4],
        [ 173,   51,    6,  ...,    1,    1,    1]]), tensor([1173,  760]))
tensor([1, 0])


In [69]:
print(batch.Text)

(tensor([[1811,   19,  404,  ...,    1,    1,    1],
        [  70, 1574,    2,  ...,  116, 1556,  100]]), tensor([ 947, 2224]))


In [56]:
# class EncoderRNN(nn.Module):
#     def __init__(self, emb_dim, h_dim, v_size, gpu=True, batch_first=True):
#         super(EncoderRNN, self).__init__()
#         self.gpu = gpu
#         self.h_dim = h_dim
#         self.embed = nn.Embedding(v_size, emb_dim)
#         self.embed.weight.data.copy_(TEXT.vocab.vectors)
#         self.lstm = nn.LSTM(emb_dim, h_dim, batch_first=batch_first,
#                             bidirectional=True)

In [57]:
class EncoderRNN(nn.Module):
    def __init__(self, emb_dim, h_dim, v_size, gpu=True, v_vec=None, batch_first=True):
        super(EncoderRNN, self).__init__()
        self.gpu = gpu
        self.h_dim = h_dim
        self.embed = nn.Embedding(v_size, emb_dim)
        if v_vec is not None:
            self.embed.weight.data.copy_(v_vec)
        self.lstm = nn.LSTM(emb_dim, h_dim, batch_first=batch_first,
                            bidirectional=True)

    def init_hidden(self, b_size):
        h0 = Variable(torch.zeros(1*2, b_size, self.h_dim))
        c0 = Variable(torch.zeros(1*2, b_size, self.h_dim))
        if self.gpu:
            h0 = h0.cuda()
            c0 = c0.cuda()
        return (h0, c0)

    def forward(self, sentence, lengths=None):
        self.hidden = self.init_hidden(sentence.size(0))
        emb = self.embed(sentence)
        packed_emb = emb

        if lengths is not None:
            lengths = lengths.view(-1).tolist()
            packed_emb = nn.utils.rnn.pack_padded_sequence(emb, lengths)

        out, hidden = self.lstm(packed_emb, self.hidden)

        if lengths is not None:
            out = nn.utils.rnn.pad_packed_sequence(output)[0]

        out = out[:, :, :self.h_dim] + out[:, :, self.h_dim:]

        return out

In [58]:
class Attn(nn.Module):
    def __init__(self, h_dim):
        super(Attn, self).__init__()
        self.h_dim = h_dim
        self.main = nn.Sequential(
            nn.Linear(h_dim, 24),
            nn.ReLU(True),
            nn.Linear(24,1)
        )

    def forward(self, encoder_outputs):
        b_size = encoder_outputs.size(0)
        attn_ene = self.main(encoder_outputs.view(-1, self.h_dim)) # (b, s, h) -> (b * s, 1)
        return F.softmax(attn_ene.view(b_size, -1), dim=1).unsqueeze(2) # (b*s, 1) -> (b, s, 1)

In [59]:
class AttnClassifier(nn.Module):
    def __init__(self, h_dim, c_num):
        super(AttnClassifier, self).__init__()
        self.attn = Attn(h_dim)
        self.main = nn.Linear(h_dim, c_num)


    def forward(self, encoder_outputs):
        attns = self.attn(encoder_outputs) #(b, s, 1)
        feats = (encoder_outputs * attns).sum(dim=1) # (b, s, h) -> (b, h)
        return F.log_softmax(self.main(feats),dim=1), attns

In [60]:
def train_model(epoch, train_iter, optimizer, log_interval=1, batch_size=2):
    encoder.train()
    classifier.train()
    correct = 0
    for idx, batch in enumerate(train_iter):
        (x, x_l), y = batch.Text, batch.Label
        optimizer.zero_grad()
        encoder_outputs = encoder(x)
        output, attn = classifier(encoder_outputs)
        loss = F.nll_loss(output, y)
        loss.backward()
        optimizer.step()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(y.data.view_as(pred)).cpu().sum()
        if idx % log_interval == 0:
            print('train epoch: {} [{}/{}], acc:{}, loss:{}'.format(
            epoch, (idx+1)*len(x), len(train_iter)*batch_size,
            correct/float(log_interval * len(x)),
            loss.data[0]))
            correct = 0

            
def test_model(epoch, test_iter):
    encoder.eval()
    classifier.eval()
    correct = 0
    for idx, batch in enumerate(test_iter):
        (x, x_l), y = batch.Text, batch.Label
        encoder_outputs = encoder(x)
        output, attn = classifier(encoder_outputs)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(y.data.view_as(pred)).cpu().sum()
    print('test epoch:{}, acc:{}'.format(epoch, correct/float(len(test))))

In [61]:
emb_dim = 300 #単語埋め込み次元
h_dim = 3 #lstmの隠れ層の次元
class_num = 2 #予測クラウ数
lr = 0.001 #学習係数
epochs = 50 #エポック数

 # make model
encoder = EncoderRNN(emb_dim, h_dim, len(TEXT.vocab),gpu=False, v_vec = TEXT.vocab.vectors)
classifier = AttnClassifier(h_dim, class_num)

# init model
def weights_init(m):
    classname = m.__class__.__name__
    if hasattr(m, 'weight') and (classname.find('Embedding') == -1):
        nn.init.xavier_uniform(m.weight.data, gain=nn.init.calculate_gain('relu'))

for m in encoder.modules():
    print(m.__class__.__name__)
    weights_init(m)
    
for m in classifier.modules():
    print(m.__class__.__name__)
    weights_init(m)

# optim
from itertools import chain
optimizer = optim.Adam(chain(encoder.parameters(),classifier.parameters()), lr=lr)

EncoderRNN
Embedding
LSTM
AttnClassifier
Attn
Sequential
Linear
ReLU
Linear
Linear


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  from ipykernel import kernelapp as app


In [62]:
# train model
for epoch in range(epochs):
    train_model(epoch + 1, train_iter, optimizer)
    test_model(epoch + 1, val_iter)

RuntimeError: invalid argument 2: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Call .contiguous() before .view(). at /Users/soumith/mc3build/conda-bld/pytorch_1549593514549/work/aten/src/TH/generic/THTensor.cpp:213

In [66]:
train.split(0.1)

(<torchtext.data.dataset.Dataset at 0x13e92d7b8>,
 <torchtext.data.dataset.Dataset at 0x13e92d748>)

In [71]:
encoder.train

<bound method Module.train of EncoderRNN(
  (embed): Embedding(5837, 300)
  (lstm): LSTM(300, 3, batch_first=True, bidirectional=True)
)>

In [72]:
len(TEXT.vocab)

5837

In [74]:
h_dim

3

In [75]:
emb_dim

300

In [78]:
TEXT.vocab.vectors.shape

torch.Size([5837, 300])